# Data Loading
Using the library pypdf extract the text from the pdf file

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader

## Extracting Questions from Pdf file

In [2]:
path: str = "data/Cuaderno_2024_BIOLOGÍA_0_C.pdf"
reader = PdfReader(path)
full_text = []
for n in range(2, len(reader.pages)):
    page = reader.pages[n]
    text: str = page.extract_text()
    full_text += text.split("\n")

Create a Data Frame of the text

In [7]:
exam_df = pd.DataFrame(full_text, columns=["text"])
exam_df.head()

,text
0,
1,Página: 1 de 19
2,1. La barrera hematoencefálica:
3,
4,1. Es permeable a todas las sustancias pre-


## Extracting Answers from .tsv file

In [4]:
answers_df = pd.read_table("data/Cuaderno_2024_BIOLOGÍA_0_C_Respuestas.tsv")
answers_df.head()

,V0,RC,V0.1,RC.1,V0.2,RC.2,V0.3,RC.3,V0.4,RC.4
0,1,2,43,1,85,2,127,4,169,3.0
1,2,2,44,4,86,1,128,1,170,1.0
2,3,3,45,3,87,2,129,2,171,2.0
3,4,1,46,3,88,4,130,3,172,2.0
4,5,2,47,1,89,4,131,1,173,1.0


# Data Cleaning

Removing rows containing "Página" and rows that have a space " "

In [12]:
exam_df_2 = exam_df[~exam_df["text"].str.contains("Página")]
exam_df_3 = exam_df_2.replace(" ", np.nan)
exam_df_3 = exam_df_3.dropna()
exam_df_3 = exam_df_3.reset_index(drop= True)
exam_df_3.head()

,text
0,1. La barrera hematoencefálica:
1,1. Es permeable a todas las sustancias pre-
2,sentes en el plasma.
3,2. Es permeable al O2 y al CO2.
4,3. Es impermeable al etanol.


In [ ]:
def process_multi_line_str(df):
    i = 0
    while i < len(df) -1:
        ...